In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

# Towns/Cities Stats Canada Urls That I would like to scrap info on
urls = [
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=cobourg&DGUIDlist=2021A00053514021&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=port%20hope&DGUIDlist=2021A00053514020&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Kingston&DGUIDlist=2021A00053510010&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Peterborough&DGUIDlist=2021A00053515014&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Lindsay&DGUIDlist=2021S05100472&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Sudbury&DGUIDlist=2021A00053553005&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Ottawa&DGUIDlist=2021A00053506008&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=orillia&DGUIDlist=2021A00053543052&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Niagara%20falls&DGUIDlist=2021A00053526043&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=st%2E%20Catharines&DGUIDlist=2021A00053526053&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=barrie&DGUIDlist=2021A00053543042&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=North%20Bay&DGUIDlist=2021A00053548044&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=London&DGUIDlist=2021A00053539036&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Stratford&DGUIDlist=2021A00053531011&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Brantford&DGUIDlist=2021A00053529006&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/Page.cfm?Lang=E&SearchText=Trenton&DGUIDlist=2021A00053512015&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Belleville&DGUIDlist=2021A00053512005&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/Page.cfm?Lang=E&SearchText=Brooklin&DGUIDlist=2021A00053518009&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/Page.cfm?Lang=E&SearchText=Bowmanville&DGUIDlist=2021A00053518017&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Burlington&DGUIDlist=2021A00053524002&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Brighton&DGUIDlist=2021A00053514004&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Brampton&DGUIDlist=2021A00053521010&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Orangeville&DGUIDlist=2021A00053522014&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/Page.cfm?Lang=E&SearchText=Bolton&DGUIDlist=2021A00053521024&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Guelph&DGUIDlist=2021A00053523008&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=kitchener&DGUIDlist=2021A00053530013&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Hamilton&DGUIDlist=2021A00053525005&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Scarborough%20Agincourt&DGUIDlist=2013A000435093&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Goderich&DGUIDlist=2021A00053540028&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Newmarket&DGUIDlist=2021A00053519048&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Oakville&DGUIDlist=2021A00053524001&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=bolton&DGUIDlist=2021S05100080&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=vaughan&DGUIDlist=2021A00053519028&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=milton&DGUIDlist=2021A00053524009&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Georgetown&DGUIDlist=2021S05100314&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Woodstock&DGUIDlist=2021A00053532042&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Chatham&DGUIDlist=2021A00053536020&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Sarnia&DGUIDlist=2021A00053538030&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Windsor&DGUIDlist=2021A00053537039&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Cambridge&DGUIDlist=2021A00053530010&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=St%20Thomas&DGUIDlist=2021A00053534021&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=nobleton&DGUIDlist=2021S05101071&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=leamington&DGUIDlist=2021A00053537003&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=fort%20erie&DGUIDlist=2021A00053526003&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Whitchurch-Stouffville&DGUIDlist=2021A00053519044&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=halton&DGUIDlist=2021A00053524015&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0&DGUIDlist=2021A00053543031&SearchText=Collingwood",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0&DGUIDlist=2021A00053519046&SearchText=Aurora",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0&DGUIDlist=2021A00053519049&SearchText=King",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0&DGUIDlist=2021A00053543074&SearchText=Midland",
    "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0&DGUIDlist=2021S05100008&SearchText=Alliston",
    
    
]

# Demographic info I would like to know abot for that city
columns = [
    "City",
    "Population, 2021",
    "Population, 2016",
    "Total private dwellings",
    "Population density per square kilometre",
    "Median after-tax income in 2020 among recipients ($)",
    "Median after-tax income of household in 2020 ($)",
    "65 years and over",
    "0 to 14 years",
    "15 to 19 years",
    "20 to 24 years",
    "0 to 24 years",
    "25 to 64 years",
    "Median age of the population",
    "Owner",
    "Bachelor's degree or higher",
    "Average household size",
    
    # White collar jobs
    "All industries",
    "51 Information and cultural industries",
    "52 Finance and insurance",
    "53 Real estate and rental and leasing",
    "54 Professional, scientific and technical services",
    "55 Management of companies and enterprises",
    "56 Administrative and support, waste management and remediation services",
    "61 Educational services",
    "62 Health care and social assistance",
    "81 Other services (except public administration)",
    "91 Public administration",
]

In [2]:
# Create a dataframe to hold the data
scrape_df = pd.DataFrame(columns=columns)

# for each url, go through each <td> and find data that matches with the columns I want
for url in urls:
    
    print(f"Scraping: {url}")
    
    # get the page html and parse it
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    
    # create an empty set to temporarly hold the data we scrape
    data = {}
    
    
    # assign the city value from the table headers
    data["City"] = (
        soup.find("table", class_="table")
        .find("thead")
        .find("tr")
        .find_all("th")[1]
        .get_text()
        .split("More")[0]
        .strip()
    )
    print(f"Scraping {data['City']}")
    
    # go through each table cell on and find columns that match the cols we want
    for td in soup.find_all("td"):
        for col in columns:
            
            # there are multiple td tags with the same title but different values
            # we just want the first so we check if the title matches and if there is a key yet
            if f"{col} - Counts - Total" in td.attrs["title"] and data.get(col) is None:
                
                # add the data to the dict but also remove any characters other than digits
                data[col] = int(re.sub('\D', "", td.get_text()))
        
    # add the data to the dataframe    
    scrape_df = pd.concat([scrape_df, pd.DataFrame(data, index=[0])], ignore_index=True)

Scraping: https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=cobourg&DGUIDlist=2021A00053514021&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0
Scraping Cobourg, Town (T)
Scraping: https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=port%20hope&DGUIDlist=2021A00053514020&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0
Scraping Port Hope, Municipality (MU)
Scraping: https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Kingston&DGUIDlist=2021A00053510010&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0
Scraping Kingston, City (CY)
Scraping: https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&SearchText=Peterborough&DGUIDlist=2021A00053515014&GENDERlist=1,2,3&STATISTIClist=1,4&HEADERlist=0
Scraping Peterborough, City (CY)
Scraping: https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E&Se

In [3]:
# calculate the percentage change in population between 2016 and 2021    
scrape_df["Population percentage change, 2016 to 2021"] = (scrape_df["Population, 2021"] - scrape_df["Population, 2016"]) / scrape_df["Population, 2021"] * 100
scrape_df["Population percentage change, 2016 to 2021"] = scrape_df["Population percentage change, 2016 to 2021"].apply(lambda num: round(num))

# we assume linear growth to extrapolate the population in 2026
scrape_df["Population, 2026"] = scrape_df["Population, 2021"] * (1 + scrape_df["Population percentage change, 2016 to 2021"] / 100)

# calculate the number of people between 0 to 24 years from the 0-14, 15-19, 20-24 years age groups
scrape_df["0 to 24 years"] = scrape_df["0 to 14 years"] + scrape_df["15 to 19 years"] + scrape_df["20 to 24 years"]

# calculate the number of people between 25 to 64 years by subtracting the 0 to 24 years and the 65 years and over age groups
scrape_df["25 to 64 years"] = scrape_df["Population, 2021"] - (scrape_df["65 years and over"] + scrape_df["0 to 24 years"])

# the columns values lost the decimal point when we scraped the info and remove special characters
# so we are adding it back
scrape_df['Median age of the population'] = scrape_df['Median age of the population'] / 10
scrape_df['Average household size'] = scrape_df['Average household size'] / 10

# we dont need these columns anymore
scrape_df.drop(['0 to 14 years', '15 to 19 years', '20 to 24 years'], axis=1, inplace=True)

# remove everything that appears after the comma and any special characters to just leave the city name
scrape_df['City'] = scrape_df['City'].apply(lambda x: x.split(",")[0])
scrape_df['City'] = scrape_df['City'].apply(lambda x: x.split("/")[0])

scrape_df['City'] = scrape_df['City'].str.replace('[^A-Za-z\s]+', '')

# Special Cases/Exceptions
# Replace Greater Sudbury with just Sudbury as thats more common name
scrape_df['City'] = scrape_df['City'].str.replace("Greater Sudbury ", "Sudbury")

# Replace ChathamKent with Chatham
scrape_df['City'] = scrape_df['City'].str.replace("ChathamKent", "Chatham")

# Replace Quinte West with Trenton (more common name)
scrape_df['City'] = scrape_df['City'].str.replace("Quinte West", "Trenton")

# Calculate the ratio of people per pharmacies
#scrape_df["Population, 2021 / pharmacy"] = scrape_df["Population, 2021"] / scrape_df["Pharmacies"]
#scrape_df["Population, 2026 / pharmacy"] = scrape_df["Population, 2026"] / scrape_df["Pharmacies"]

# save the results
scrape_df.to_csv("./csv/census_canada_scrape.csv", index=False)

C:\Users\shann\AppData\Local\Temp\ipykernel_11208\3454191721.py:26: FutureWarning: The default value of regex will change from True to False in a future version.
  scrape_df['City'] = scrape_df['City'].str.replace('[^A-Za-z\s]+', '')
